In [1]:
## Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import os
# import glob
# import sys
import ast

# set inline print
%matplotlib inline

rootdir = os.path.join(".")
path_logs = os.path.join(rootdir,"learning_logs.csv")

def apfd(nqueries: list, hypsize: list, max_nqueries=None):
    hs = hypsize
    nq = nqueries
    assert hs[0] == 1
    assert len(nq) == len(hs)
    
    extra_nqueries, extra_hs = [],[]
    if(not max_nqueries is None): 
        extra_nqueries = [max_nqueries]
        extra_hs = [hs[-1]]

    return 1 - (np.sum(np.multiply([*nq,*extra_nqueries], np.diff([0, *hs,*extra_hs]))) / (np.max([*nq,*extra_nqueries]) * np.max(hs))) + (1.0 / (2 * np.max([*nq,*extra_nqueries])))

def auc_learning(nqueries: list, hypsize: list):
    hs = hypsize
    nq = nqueries
    assert hs[0] == 1
    assert len(nq) == len(hs)
    return np.trapz(hs,nq)

def derive_data(data_frame: pd.DataFrame.dtypes):
    # first, copy dataframe
    df = data_frame.copy()    
    # ... and then split queries/symbols into different columns
    for qtype in ["Learning", "Testing"]:
        _lst= df[f"{qtype} queries/symbols"].apply(lambda x: [i.split('/') for i in ast.literal_eval(x)])
        df[f"{qtype} queries"]            = _lst.apply(lambda x : np.cumsum([int(i[0]) for i in x])) # resets
        df[f"{qtype} symbols"]            = _lst.apply(lambda x : np.cumsum([int(i[1]) for i in x])) # symbols w/o resets

    # ... and then parsing string with hypotheses sizes as array of integers
    df["HypSize"] = df["HypSize"].apply(lambda x: ast.literal_eval(x)) 
    df["TQ [Symbols]"] = df["EQ [Symbols]"]+df["MQ [Symbols]"]
    df["TQ [Resets]"] = df["EQ [Resets]"]+df["MQ [Resets]"]

    # ... and then append qSize to HypSize, if the run is successfull 
    df["HypSize"] = df.apply(lambda x: x.HypSize + [x.Qsize] if x.Equivalent=='OK' and len(x.HypSize) < x.Rounds else x.HypSize, axis=1)
    
    # ... and then include #EQs from the single-state model
    df["Testing queries"] = df["Testing queries"].apply(lambda x: [0,*x])
    df["Testing symbols"] = df["Testing symbols"].apply(lambda x: [0,*x])
    
    # ... and then calculate the total number of queries
    df["Total queries"] = df.apply(lambda x: np.add(x["Testing queries"],x["Learning queries"]) if x.Equivalent=='OK' else [], axis=1)
    df["Total symbols"] = df.apply(lambda x: np.add(x["Testing symbols"],x["Learning symbols"]) if x.Equivalent=='OK' else [], axis=1)
    
    # ... and then (FINALLY!) calculate the APFD and AUC for EQs, and TQs
    df_eq = df.query('`Equivalent`=="OK"')
    
    the_cols = ["SUL name","TQ [Symbols]","EQ [Symbols]"]
    max_eqs = df[the_cols].groupby(["SUL name"]).max().to_dict()

    df["APFD_testing"] = df.apply(lambda x: apfd(x['Testing symbols'],x['HypSize']) if x.Equivalent=='OK' else -1, axis=1)
    df["APFD_total"] = df.apply(lambda x: apfd(x['Total symbols'],x['HypSize']) if x.Equivalent=='OK' else -1, axis=1)
    
    df["APFDx_testing"] = df.apply(lambda x: apfd(x['Testing symbols'],x['HypSize'],max_nqueries=max_eqs['EQ [Symbols]'][x['SUL name']]) if x.Equivalent=='OK' else -1, axis=1)
    df["APFDx_total"] = df.apply(lambda x: apfd(x['Total symbols'],x['HypSize'],max_nqueries=max_eqs['TQ [Symbols]'][x['SUL name']]) if x.Equivalent=='OK' else -1, axis=1)
    
    df["AUC_testing"] = df.apply(lambda x: auc_learning(x['Testing symbols'],x['HypSize']) if x.Equivalent=='OK' else -1, axis=1)
    df["AUC_total"] = df.apply(lambda x: auc_learning(x['Total symbols'],x['HypSize']) if x.Equivalent=='OK' else -1, axis=1)
    
    # to close, return the new dataframe with derived columns
    return df

# Split columns with queries/symbols

In [2]:
df = pd.read_csv(path_logs)
df = derive_data(df)
df.columns

Index(['Method', 'Seed', 'EquivalenceOracle', 'CTT', 'Extra States',
       'Random Infix Length', 'Cache', 'SUL name', 'Qsize', 'Isize',
       'TreeType', 'Learning queries/symbols', 'Testing queries/symbols',
       'HypSize', 'OTreeSize', 'MQ [Resets]', 'MQ [Symbols]', 'EQ [Resets]',
       'EQ [Symbols]', 'Rounds', 'Learning [ms]', 'Equivalent',
       'Learning queries', 'Learning symbols', 'Testing queries',
       'Testing symbols', 'TQ [Symbols]', 'TQ [Resets]', 'Total queries',
       'Total symbols', 'APFD_testing', 'APFD_total', 'APFDx_testing',
       'APFDx_total', 'AUC_testing', 'AUC_total'],
      dtype='object')

In [3]:
sort_by=['APFDx_testing']
bitwise=df.query('`Equivalent`=="OK" and `SUL name`=="DropBear.dot"').sort_values(by= sort_by)
bitwise[['SUL name', 'Method', 'CTT', 'Extra States', 'AUC_testing', *sort_by]]

,SUL name,Method,CTT,Extra States,AUC_testing,APFDx_testing
287,DropBear.dot,"L# (SepSeq, SepSeq)",W,2,4525481.0,0.581658
52,DropBear.dot,"L# (SepSeq, SepSeq)",Wp,2,3799340.0,0.672705
283,DropBear.dot,"L# (SepSeq, SepSeq)",HadsInt,2,4465538.5,0.721056
190,DropBear.dot,"L# (SepSeq, SepSeq)",Hsi,2,3572657.5,0.763579
177,DropBear.dot,"L# (SepSeq, SepSeq)",W,1,427252.5,0.969381
363,DropBear.dot,"L# (SepSeq, SepSeq)",Wp,1,300033.5,0.972374
331,DropBear.dot,"L# (SepSeq, SepSeq)",HadsInt,1,346150.0,0.980813
367,DropBear.dot,"L# (SepSeq, SepSeq)",Hsi,1,289306.5,0.983445


In [4]:
df_equiv=df.query(f'`Equivalent`=="OK"').sort_values(by=['APFDx_testing'],ascending=False)

# for qtype in ['Testing symbols', 'Total symbols']:
for qtype in ['Testing symbols', 'Total symbols', 'Testing queries', 'Total queries']:
# for qtype in ['Testing']:
    print(qtype)
    for sul_name in df['SUL name'].drop_duplicates().sort_values():
        print('\t',sul_name)
        # get one entry
        subj=df_equiv.query(f'`SUL name`=="{sul_name}"').copy() 

        # add percent columns
        subj['HypSizePercent'] = subj['HypSize'].apply(lambda x: x/np.max(x)*100)
        subj['learningsetup']  = subj['CTT']+"("+subj['Extra States'].astype(str)+")"

        # explode column with % of symbols and hypothesis sizes in the learning process
        subj=subj.explode(['HypSizePercent',f'{qtype}'])
        
        # define figure size
        sns.set(rc={'figure.figsize':(10,5),'figure.dpi':200})
        
        #create line chart
        apfd_plot = sns.lineplot(subj, x=f'{qtype}', y='HypSizePercent',
                                 markers=True, 
                                 #style='Extra States', hue='CTT',
                                 style='learningsetup', hue='learningsetup',
                                 palette='rocket'
                                )
        apfd_plot.set(xscale='log')
        locator = ticker.LogLocator()
        locator.MAXTICKS = np.max(subj[f'{qtype}'])
        apfd_plot.xaxis.set_major_locator(locator)
        
        apfd_plot.yaxis.set_major_locator(ticker.MultipleLocator(10))
        apfd_plot.set_ylim(0,100)
        
        #add axis labels
        plt.xlabel(f'#{qtype} Posed')
        plt.ylabel('% of States Discovered')
        plt.legend(title='Testing Technique', loc='lower right')
        plt.title(f'{sul_name}')
        
        ##display line chart
        #plt.show()
        
        # save line chart
        fig = apfd_plot.get_figure()
        
        os.makedirs(f'img/cumulative/{qtype}/', exist_ok=True)
        fig.savefig(f'img/cumulative/{qtype}/'+sul_name.replace('.dot',f'_cumulative_{qtype}.jpg'))
        fig.clf()

Testing symbols
	 10_learnresult_MasterCard_fix.dot
	 1_learnresult_MasterCard_fix.dot
	 4_learnresult_MAESTRO_fix.dot
	 4_learnresult_PIN_fix.dot
	 4_learnresult_SecureCode Aut_fix.dot
	 ASN_learnresult_MAESTRO_fix.dot
	 ASN_learnresult_SecureCode Aut_fix.dot
	 BitVise.dot
	 DropBear.dot
	 GnuTLS_3.3.12_client_full.dot
	 GnuTLS_3.3.12_client_regular.dot
	 GnuTLS_3.3.12_server_full.dot
	 GnuTLS_3.3.12_server_regular.dot
	 GnuTLS_3.3.8_client_full.dot
	 GnuTLS_3.3.8_client_regular.dot
	 GnuTLS_3.3.8_server_full.dot
	 GnuTLS_3.3.8_server_regular.dot
	 NSS_3.17.4_client_full.dot
	 NSS_3.17.4_client_regular.dot
	 NSS_3.17.4_server_regular.dot
	 OpenSSH.dot
	 OpenSSL_1.0.1g_client_regular.dot
	 OpenSSL_1.0.1g_server_regular.dot
	 OpenSSL_1.0.1j_client_regular.dot
	 OpenSSL_1.0.1j_server_regular.dot
	 OpenSSL_1.0.1l_client_regular.dot
	 OpenSSL_1.0.1l_server_regular.dot
	 OpenSSL_1.0.2_client_full.dot
	 OpenSSL_1.0.2_client_regular.dot
	 OpenSSL_1.0.2_server_regular.dot
	 RSA_BSAFE_C_4.0.4_s

<Figure size 2000x1000 with 0 Axes>

In [5]:
def auc_addsorted(alist, datapoints=[]):
    cc_dp = alist.copy()
    for newdp in datapoints:
        if(newdp in cc_dp): continue
        cc_dp = np.insert(cc_dp,np.searchsorted(cc_dp,newdp),newdp)
    return cc_dp

def auc_apfd(data: pd.DataFrame.dtypes, col_costs: str, col_hypsizes: str, datapoints=[]):
    df_subset = data.copy()
    df_subset[col_hypsizes+'_withdatapoints']=df_subset[col_hypsizes].apply(lambda x: auc_addsorted(x,datapoints))
    df_subset[col_costs]=df_subset.apply(lambda x: np.interp(x[col_hypsizes+'_withdatapoints'], x[col_hypsizes], x[col_costs]),axis=1)
    df_subset[col_hypsizes]=df_subset[col_hypsizes+'_withdatapoints']
    df_subset.drop(col_hypsizes+'_withdatapoints',inplace=True,axis=1)
    return df_subset

new_dps = list(range(10,101,10))

df_equiv=df.query(f'`Equivalent`=="OK"').sort_values(by=['APFDx_testing'],ascending=False)
for qtype in ['Testing symbols', 'Total symbols', 'Testing queries', 'Total queries']:
# for qtype in ['Testing']:
    print(qtype)
    for sul_name in df['SUL name'].drop_duplicates().sort_values():
        print('\t',sul_name)
        # get one entry
        subj=df_equiv.query(f'`SUL name`=="{sul_name}"').copy() 

        # add percent columns
        subj['HypSizePercent'] = subj['HypSize'].apply(lambda x: x/np.max(x)*100)
        subj['CTT_ES']  = subj['CTT']+"("+subj['Extra States'].astype(str)+")"

        subj_interp=auc_apfd(data=subj, col_costs=f'{qtype}',col_hypsizes='HypSizePercent', datapoints=new_dps)
        subj_interp=subj_interp.explode([f'{qtype}','HypSizePercent'])
        subj_interp=subj_interp.query('HypSizePercent in @new_dps').sort_values(by=[f'{qtype}'],ascending=False)

        cols_order = subj_interp['CTT_ES'].drop_duplicates().to_list()
        cols_order.reverse()

        the_cols = ['SUL name','HypSizePercent','CTT_ES']
        subj_gb=subj_interp[the_cols+[f'{qtype}',]].groupby(the_cols).first()

        ## The scope of these changes made to
        ## pandas settings are local to with statement.
        #with pd.option_context('display.max_rows', None,
        #                       'display.max_columns', None,
        #                       'display.precision', 3,
        #                       'display.float_format', '{:,.1f}'.format,
        #                       ):
        #    next
        #    display(subj_gb.pivot_table(f'{qtype}','HypSizePercent',['SUL name','CTT_ES'])[[['BitVise.dot',it] for it in cols_order]])
        #    display(subj_gb)
        
        # define figure size
        sns.set(rc={'figure.figsize':(20,5)})
        
        subj_pvt = subj_gb.pivot_table(f'{qtype}',['CTT_ES'],'HypSizePercent').sort_values(by=[100.0])
        subj_pvt = subj_pvt/subj_pvt.max().max()
        heatmap_cost=sns.heatmap(subj_pvt,
                                 #annot=True, fmt=",.7f", 
                                 cmap= sns.cm.rocket_r, linewidth=.5, linecolor='black'
                                )
        
        ##display line chart
        #plt.show()
        
        # save line chart
        fig = heatmap_cost.get_figure()
        os.makedirs(f'img/heatmap/{qtype}/', exist_ok=True)
        fig.savefig(f'img/heatmap/{qtype}/'+sul_name.replace('.dot',f'_heatmap_{qtype}.jpg'))
        fig.clf()

Testing symbols
	 10_learnresult_MasterCard_fix.dot
	 1_learnresult_MasterCard_fix.dot
	 4_learnresult_MAESTRO_fix.dot
	 4_learnresult_PIN_fix.dot
	 4_learnresult_SecureCode Aut_fix.dot
	 ASN_learnresult_MAESTRO_fix.dot
	 ASN_learnresult_SecureCode Aut_fix.dot
	 BitVise.dot
	 DropBear.dot
	 GnuTLS_3.3.12_client_full.dot
	 GnuTLS_3.3.12_client_regular.dot
	 GnuTLS_3.3.12_server_full.dot
	 GnuTLS_3.3.12_server_regular.dot
	 GnuTLS_3.3.8_client_full.dot
	 GnuTLS_3.3.8_client_regular.dot
	 GnuTLS_3.3.8_server_full.dot
	 GnuTLS_3.3.8_server_regular.dot
	 NSS_3.17.4_client_full.dot
	 NSS_3.17.4_client_regular.dot
	 NSS_3.17.4_server_regular.dot
	 OpenSSH.dot
	 OpenSSL_1.0.1g_client_regular.dot
	 OpenSSL_1.0.1g_server_regular.dot
	 OpenSSL_1.0.1j_client_regular.dot
	 OpenSSL_1.0.1j_server_regular.dot
	 OpenSSL_1.0.1l_client_regular.dot
	 OpenSSL_1.0.1l_server_regular.dot
	 OpenSSL_1.0.2_client_full.dot
	 OpenSSL_1.0.2_client_regular.dot
	 OpenSSL_1.0.2_server_regular.dot
	 RSA_BSAFE_C_4.0.4_s

<Figure size 4000x1000 with 0 Axes>